### Data Ingestion 

In [1]:
### Document Structure 

from langchain_core.documents import Document

In [3]:
doc  = Document(
    page_content='this is the main text content I am using for RAG',
    metadata= {
        "source": "example.txt",
        "pages": 1,
        "author":"Batman",
        "date_created": "2025-01-01"
    }
)
## This is helpful in applying filters to search things by , talking of metadata 

In [5]:
## create a txt file 
import os 
os.makedirs("../data/text_files", exist_ok=True)


In [6]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


In [11]:
### TextLoader 
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader 
# This shenanigan is to use whatever is not deprecatedl. 


loader = TextLoader("../data/text_files/python_intro.txt", encoding= "utf-8")
#
#If a class does not define a special __str__ method, Python falls back to __repr__.

# Many libraries (including langchain) don’t bother implementing both, so str(loader) and repr(loader) return the same string.
###
document = loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [ ]:
### Directory loader 
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the dierctory 
dir_loader = DirectoryLoader(
    "../data/text_files", 
    glob= "**/*.txt",
    loader_cls = TextLoader, ### loader class to use 
    loader_kwargs = {'encoding': "utf-8"}, 
    show_progress= False
)

dir_loader.load()

To log the progress of DirectoryLoader you need to install tqdm, `pip install tqdm`


ImportError: To log the progress of DirectoryLoader you need to install tqdm, `pip install tqdm`